In [1]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import json
import re
import sqlite3 
import pandas as pd

In [2]:
from tensorflow.keras.preprocessing.text import tokenizer_from_json

with open('tokenizer.json') as f:
    data = json.load(f)
    tokenizer = tokenizer_from_json(data)

In [3]:
loaded_model = load_model(r'Model_LSTM.h5')

In [4]:
#Ubah text menjadi lower
def lower(text):
    return text.lower()

In [5]:
def hapuskarakter(text):
    text = re.sub('\n',' ', text)
    text = re.sub('rt',' ', text)
    text = re.sub('user',' ', text)
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+)|(http?://[^\s]+))',' ',text)
    text = re.sub('  +',' ', text)
    return text

In [6]:
db = sqlite3.connect('dbp.db', check_same_thread=False)
db.text_factory = bytes
mycursor = db.cursor()

#Baca Table Kamus Alay
query = "select * from kamus_alay"
kamusalay = pd.read_sql_query(query, db)
kamusalay['hasil clean'] = kamusalay['hasil clean'].str.decode('utf-8')
kamusalay['kata alay'] = kamusalay['kata alay'].str.decode('utf-8')

alay_dict_map = dict(zip(kamusalay['kata alay'], kamusalay['hasil clean']))
def replace_kamus_alay(text):
    for word in alay_dict_map:
        return ' '.join([alay_dict_map[word] if word in alay_dict_map else word for word in text.split(' ')])

In [7]:
def cleansing(text):
    text = lower(text)
    text = hapuskarakter(text)
    text = replace_kamus_alay(text)
    return text

In [8]:
def pred_sentiment(string):    
    string = cleansing(string)

    sekuens_x = tokenizer.texts_to_sequences([string])
    padded_x = pad_sequences(sekuens_x)

    classes = loaded_model.predict(padded_x, batch_size=10)

    return classes[0]

In [9]:
def pred(classes):
    if classes[0] == classes.max():
        print('negative')
    if classes[1] == classes.max():
        print('neutral')
    if classes[2] == classes.max():
        print('positive')

In [10]:
string = "Dasar Kamu bodoh!"

In [11]:
classes = pred_sentiment(string)
pred(classes)

1/1 [==============================] - 1s 872ms/step
negative


In [12]:
string = "Binar Academy merupakan lembaga pendidikan yang menyelenggarakan program kursus dalam bidang digital"

In [13]:
classes = pred_sentiment(string)
pred(classes)

1/1 [==============================] - 1s 656ms/step
neutral


In [14]:
string = "setiap bumn dibentuk dengan uu bukan dibentuk dengan uu"

In [15]:
classes = pred_sentiment(string)
pred(classes)

1/1 [==============================] - 0s 31ms/step
neutral


In [16]:
string = "km jelek bgt sich!"

In [17]:
classes = pred_sentiment(string)
pred(classes)

1/1 [==============================] - 0s 31ms/step
negative
